In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random
from random import shuffle
from IPython.display import Image, display
from tqdm import tqdm
from tensorflow.python import keras
DIR = 'C:/Users/Leo/Desktop/Cambridge/Probably_right_notes/Computer_Vision/mini-project/tiny-imagenet-200/tiny-imagenet-200'
file = os.listdir(DIR)
print(file)

img_rows, img_cols = 64,64
num_classes = 200

['test', 'train', 'val', 'wnids.txt', 'words.txt']


In [2]:
from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
#from keras.applications.imagenet_utils import obtain_input_shape
from keras.engine.topology import get_source_inputs
from keras.layers import merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D,ZeroPadding2D,AveragePooling2D
from keras.layers.core import Dense, Activation,Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers import Input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import concatenate
from keras.layers import Dropout
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D
from keras.callbacks import Callback


Using TensorFlow backend.


In [3]:
def block1(x, nb_filter, kernel_size=3):
    k1,k2,k3 = nb_filter
    out = Convolution2D(k1,1,1)(x)
    out = BatchNormalization()(out)
    out = LeakyReLU(alpha=0.01)(out)
    #out = Activation('relu')(out)
    
    out = Convolution2D(k2,kernel_size,kernel_size,border_mode='same')(out)
    out = BatchNormalization()(out)
    out = LeakyReLU(alpha=0.01)(out)
    #out = Activation('relu')(out)
    
    out = Convolution2D(k3,1,1)(out)
    out = BatchNormalization()(out)
    
    out = layers.add([out,x])
    out = LeakyReLU(alpha=0.01)(out)
    
    return out

def block2(x, nb_filter, kernel_size=3):
    k1,k2,k3 = nb_filter
    out = Convolution2D(k1,1,1)(x)
    out = BatchNormalization()(out)
    out = LeakyReLU(alpha=0.01)(out)
    #out = Activation('relu')(out)
    
    out = Convolution2D(k2,kernel_size,kernel_size,border_mode='same')(out)
    out = BatchNormalization()(out)
    out = LeakyReLU(alpha=0.01)(out)
    #out = Activation('relu')(out)
    
    out = Convolution2D(k3,1,1)(out)
    out = BatchNormalization()(out)
    
        
    x = Convolution2D(k3,1,1)(x)
    x = BatchNormalization()(x)

    out = layers.add([out,x])
    out = LeakyReLU(alpha=0.01)(out)
    
    return out

In [4]:
train_x = np.load('train_x12.npy')
train_y = np.load('train_y12.npy')

In [5]:
print(train_x.shape)

(200000, 64, 64, 3)


In [6]:
train_xx = np.load('train_x3.npy')
train_yx = np.load('train_y3.npy')
print(train_xx.shape)

(100000, 64, 64, 3)


In [ ]:
def ResNet8():
    
    inp = Input(shape=(img_rows,img_cols,3))
    out = ZeroPadding2D((3,3))(inp)
    out = Convolution2D(64,7,7,subsample=(2,2))(inp)
    out = BatchNormalization()(out) # same function as dropout
    out = LeakyReLU(alpha=0.01)(out)
    out = MaxPooling2D((3,3),strides=(2,2))(out)
    
   
    out = block2(out,[256,256,1024])
    out = block1(out,[256,256,1024])
    out = block1(out,[256,256,1024])
    out = block1(out,[256,256,1024])

    
    
    out = AveragePooling2D((7,7))(out)
    out = Flatten()(out)
    out = Dense(200, activation='softmax')(out)
    
    
    model = Model(inp,out)
    
    return model

model = ResNet8()
model.compile(loss='poisson',optimizer='adam',metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=100,epochs=2,verbose=2,validation_split=0.05)

c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2))`
  """
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1))`
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1))`
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1))`
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykern

Train on 142646 samples, validate on 7508 samples
Epoch 1/2


In [9]:
model.compile(loss='poisson',optimizer='adam',metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=100,epochs=10,verbose=2,validation_split=0.05)

Train on 142646 samples, validate on 7508 samples
Epoch 1/10
 - 942s - loss: 0.0190 - acc: 0.3531 - val_loss: 0.0200 - val_acc: 0.3181
Epoch 2/10
 - 929s - loss: 0.0170 - acc: 0.4281 - val_loss: 0.0199 - val_acc: 0.3247
Epoch 3/10
 - 904s - loss: 0.0154 - acc: 0.4935 - val_loss: 0.0193 - val_acc: 0.3522
Epoch 4/10
 - 904s - loss: 0.0140 - acc: 0.5516 - val_loss: 0.0171 - val_acc: 0.4254
Epoch 5/10
 - 907s - loss: 0.0128 - acc: 0.6045 - val_loss: 0.0175 - val_acc: 0.4330
Epoch 6/10
 - 910s - loss: 0.0116 - acc: 0.6564 - val_loss: 0.0174 - val_acc: 0.4362
Epoch 7/10
 - 919s - loss: 0.0105 - acc: 0.7079 - val_loss: 0.0170 - val_acc: 0.4603
Epoch 8/10
 - 935s - loss: 0.0094 - acc: 0.7572 - val_loss: 0.0172 - val_acc: 0.4635
Epoch 9/10
 - 962s - loss: 0.0085 - acc: 0.8052 - val_loss: 0.0175 - val_acc: 0.4694
Epoch 10/10
 - 968s - loss: 0.0077 - acc: 0.8443 - val_loss: 0.0180 - val_acc: 0.4736


In [10]:
model.compile(loss='poisson',optimizer='adam',metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=100,epochs=5,verbose=2,validation_split=0.05)

Train on 142646 samples, validate on 7508 samples
Epoch 1/5
 - 979s - loss: 0.0072 - acc: 0.8740 - val_loss: 0.0180 - val_acc: 0.4828
Epoch 2/5
 - 982s - loss: 0.0066 - acc: 0.9043 - val_loss: 0.0183 - val_acc: 0.4826
Epoch 3/5
 - 992s - loss: 0.0063 - acc: 0.9219 - val_loss: 0.0186 - val_acc: 0.4812
Epoch 4/5
 - 974s - loss: 0.0061 - acc: 0.9349 - val_loss: 0.0196 - val_acc: 0.4702
Epoch 5/5
 - 976s - loss: 0.0059 - acc: 0.9439 - val_loss: 0.0193 - val_acc: 0.4988


In [ ]:
model.save('ResNet14.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

In [5]:
model = load_model('ResNet14.h5') # 

In [6]:
model.compile(loss='poisson',optimizer='adam',metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=100,epochs=3,verbose=2,validation_split=0.05)

Train on 142646 samples, validate on 7508 samples
Epoch 1/3
 - 1012s - loss: 0.0059 - acc: 0.9461 - val_loss: 0.0202 - val_acc: 0.4879
Epoch 2/3
 - 948s - loss: 0.0057 - acc: 0.9558 - val_loss: 0.0196 - val_acc: 0.5003
Epoch 3/3
 - 1033s - loss: 0.0057 - acc: 0.9605 - val_loss: 0.0205 - val_acc: 0.4871


In [7]:
def ResNet8():
    
    inp = Input(shape=(img_rows,img_cols,3))
    print(inp.shape)
    out = ZeroPadding2D((3,3))(inp)
    out = Convolution2D(64,7,7,subsample=(2,2))(inp)
    out = BatchNormalization()(out) # same function as dropout
    out = LeakyReLU(alpha=0.01)(out)
    out = MaxPooling2D((3,3),strides=(2,2))(out)
    print(out.shape)
   
    out = block2(out,[256,256,1024])
    print(out.shape)
    out = block1(out,[256,256,1024])
    out = block1(out,[256,256,1024])
    out = block1(out,[256,256,1024])
    
    
    out = AveragePooling2D((7,7))(out)
    out = Flatten()(out)
    out = Dense(200, activation='softmax')(out)
    
    
    model = Model(inp,out)
    
    return model

model = ResNet8()
model.compile(loss='poisson',optimizer='adam',metrics=['accuracy'])
model.fit(train_x,train_y,batch_size=100,epochs=20,verbose=2,validation_split=0.05)

(?, 64, 64, 3)


c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2))`
  


(?, 14, 14, 64)


c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1))`
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1))`
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1))`


(?, 14, 14, 1024)


c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1))`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
  
c:\users\leo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (1, 1))`
  del sys.path[0]


NameError: name 'train_x' is not defined

In [ ]:
model.fit(train_xx,train_yx,batch_size=100,epochs=20,verbose=2,validation_split=0.05)

Train on 95000 samples, validate on 5000 samples
Epoch 1/20
 - 648s - loss: 0.0190 - acc: 0.3550 - val_loss: 0.0183 - val_acc: 0.3772
Epoch 2/20
 - 553s - loss: 0.0142 - acc: 0.5370 - val_loss: 0.0173 - val_acc: 0.4374
Epoch 3/20
 - 552s - loss: 0.0117 - acc: 0.6529 - val_loss: 0.0177 - val_acc: 0.4322
Epoch 4/20
 - 552s - loss: 0.0096 - acc: 0.7538 - val_loss: 0.0177 - val_acc: 0.4462
Epoch 5/20


In [9]:
model.fit(train_xx,train_yx,batch_size=100,epochs=20,verbose=2,validation_split=0.05)

Train on 95000 samples, validate on 5000 samples
Epoch 1/20
 - 553s - loss: 0.0058 - acc: 0.9552 - val_loss: 0.0224 - val_acc: 0.4302
Epoch 2/20
 - 554s - loss: 0.0056 - acc: 0.9668 - val_loss: 0.0219 - val_acc: 0.4408
Epoch 3/20
 - 553s - loss: 0.0056 - acc: 0.9682 - val_loss: 0.0229 - val_acc: 0.4276
Epoch 4/20
 - 553s - loss: 0.0055 - acc: 0.9711 - val_loss: 0.0232 - val_acc: 0.4272
Epoch 5/20
 - 564s - loss: 0.0055 - acc: 0.9751 - val_loss: 0.0236 - val_acc: 0.4306
Epoch 6/20


KeyboardInterrupt: 

In [10]:
def test_acc(model,datax,datay):
    preds = model.predict(datax)
    pred_index = np.argmax(preds, axis=1)
    trainlabel = np.argmax(datay,axis=1)
    bolcom = np.equal(trainlabel,pred_index)
    test_accuracy = sum(bolcom)/pred_index.shape*100
    print('Test Accuracy',test_accuracy)
    return test_accuracy

In [13]:
test_acc(model,train_x[:],train_y[:])

Test Accuracy [40.324]


array([40.324])